# time series comparisons of QPEs at specific sites

This notebook contains code to visually compare the time series of hourly intensities reported by different gridded QPE products for a given location and storm

Code in this notebookis written assuming that the QPE data has already been cleaned using the cleaning scripts provided (or, equivalently, that all QPE data is stored in netCDF files containing hourly precipitation intensities, with coordinates of 'time', 'lat', and 'lon')

Lines with comments that start with "USER INPUT!" are places that will likely need changing, based on your time period, location, and QPEs of interest

### import libraries

In [1]:
# import libraries
import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats, signal
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature

### setup

In [2]:
dataset_path = '/path/to/cleaned_data/'     # USER INPUT! path to cleaned QPE datasets

storm_start = '2021-01-27T01:00'            # USER INPUT! start of time period (string format, YYYY-MM-DDTHH:MM)
storm_end = '2021-01-30T00:00'              # USER INPUT! start of time period (string format, YYYY-MM-DDTHH:MM)

hrrr_fh = 1                                 # USER INPUT! if using HRRR model, forecast hour of model data

In [ ]:
gauge_times =                               # USER INPUT! list, series, array, etc. of gauge reading times
gauge_intensities =                         # USER INPUT! list, series, array, etc. of gauge intensities
# note: for the most fair comparison, I would recommend using hourly gauge data
# you can resample gauge data to hourly using the resample() function in resample_qc.py :)
# the next cell is an example of how I call the data I downloaded using the synoptic_download_qc notebook

### QPEs to use

In [2]:
# USER INPUT! change lists to meet your data needs

qpe_list = [                   
    'mrmsro', 
    'mrms', 
    'nldas2', 
    'cmorph', 
    'imerg', 
    'gsmap', 
    'persiann', 
    'he',    
    'era5', 
    'era5land', 
    'merra2', 
    'hrrr'
]

qpe_name_list = [
    'MRMS-RO', 
    'MRMS-MS', 
    'NLDAS-2', 
    'CMORPH', 
    'IMERG', 
    'GSMaP', 
    'PERSIANN-CCS', 
    'HE',  
    'ERA5', 
    'ERA5-Land', 
    'MERRA-2', 
    'HRRR'
]

### define function for finding QPE grid cell co-located with coordinates

In [ ]:
def find_grid_cell(input_lat, input_lon, grid_lats, grid_lons):
    """Finds the grid cell of a gridded QPE closest to an input lat and lon

    input_lat (float): latitude of location of interest
    input_lon (float): longitude of location of interest
    grid_lats (array): latitude coordinate values of gridded products
    grid_lats (array): longitude coordinate values of gridded products

    Returns:
    EITHER lat_idx, lon_idx (for rectilinear products) OR 
    x_idx, y_idx (for curvilinear products): indices that can be used to
    find cell containing input coordinates 

    """
    # if on a rectilinear grid
    if grid_lats.ndim == 1:
        lat_idx = np.abs(grid_lats - input_lat).argmin()
        lon_idx = np.abs(grid_lons - input_lon).argmin()
        return lat_idx, lon_idx
    
    # if on a curvilinear grid
    elif grid_lats.ndim == 2:
        idx_flat = np.sqrt((grid_lats - input_lat)**2 + (grid_lons - input_lon)**2).argmin()
        x_idx, y_idx = np.unravel_index(idx_flat, grid_lats.shape)
        return x_idx, y_idx

### define time arrays using storm start and stop times set above

In [4]:
# define list of hours in time period of interest
time_array = np.arange(storm_start, storm_end, dtype='datetime64[h]') 
time_array_hrrr = time_array - np.timedelta64(hrrr_fh, 'h') # adjusted hours for forecast model

### import QPE data

In [ ]:
# loop over cleaned QPE datasets
for qpe in qpe_list:
    # import
    da = xr.open_dataarray(f'{dataset_path}/{qpe}.nc')
    # crop to time period of interest
    if (qpe=='hrrr'):
        da = da.sel(time=slice(time_array_hrrr[0], time_array_hrrr[-1]))
    else:
        da = da.sel(time=slice(time_array[0], time_array[-1]))
    # save dataarray for plotting/analysis
    globals()[f'{qpe}_da'] = da

### set up for comparing data at specific location

In [ ]:
ls_coords = (,)              # USER INPUT! coordinates of landslide of interest
gauge_coords = (,)           # USER INPUT! coordinates of nearest gauge
ls_time_known = True.        # USER INPUT! is the time of landslide known?
ls_time = 'YYYY-MM-DDTHH:MM' # USER INPUT! time of landslide (if known), string YYYY-MM-DDTHH:MM
plot_spread = True           # USER INPUT! whether or not to show spread among QPEs on figure
save_gauge_fig = False       # USER INPUT! whether or not to save time series at gauge location
save_ls_fig = False          # USER INPUT! whether or not to save time series at landslide location

### find grid cells co-located with desired location

In [ ]:
# loop through QPEs
for qpe in qpe_list:
    qpe_da = globals()[f'{qpe}_da']
    qpe_lat = qpe_da.lat.values
    qpe_lon = qpe_da.lon.values

    
    # find cell at gauge location 
    g_cell_idx = find_grid_cell(gauge_coords[0], gauge_coords[1], qpe_lat, qpe_lon)
    if not all(x in qpe_da.dims for x in ['lat', 'lon']):
        qpe_g_lat = qpe_lat[g_cell_idx]
        qpe_g_lon = qpe_lon[g_cell_idx]
        g_cell_da = qpe_da.where((qpe_da.lat==qpe_g_lat) & (qpe_da.lon==qpe_g_lon), drop=True)[:, 0, 0]
    else:
        qpe_g_lat = qpe_lat[g_cell_idx[0]]
        qpe_g_lon = qpe_lon[g_cell_idx[1]]
        g_cell_da = qpe_da.sel(lat=qpe_g_lat, lon=qpe_g_lon, method='nearest')

    globals()[f'{qpe}_gauge_da'] = g_cell_da
    
    
    # find cell at landslide location 
    ls_idx = find_grid_cell(ls_coords[0], ls_coords[1], qpe_lat, qpe_lon)
    if not all(x in qpe_da.dims for x in ['lat', 'lon']):
        qpe_ls_lat = qpe_lat[ls_idx]
        qpe_ls_lon = qpe_lon[ls_idx]
        ls_cell_da = qpe_da.where((qpe_da.lat==qpe_ls_lat) & (qpe_da.lon==qpe_ls_lon), drop=True)[:, 0, 0]
    else:
        qpe_ls_lat = qpe_lat[ls_idx[0]]
        qpe_ls_lon = qpe_lon[ls_idx[1]]
        ls_cell_da = qpe_da.sel(lat=qpe_ls_lat, lon=qpe_ls_lon, method='nearest')

    globals()[f'{qpe}_ls_da'] = ls_cell_da

### plot the time series at the GAUGE location

In [ ]:
# create figure
fig, ax = plt.subplots(1, 1, figsize=(18, 8))

# calculate and plot spread among QPEs at each hour
if plot_spread == True:
    mins = []
    maxs = []
    # calculate
    for hour in time_array:
        int_list = [globals()[f'{qpe}_gauge_da'].sel(time=hour, method='nearest').values for qpe in qpe_list]
        int_list.append(gauge.sel(time=hour).values)
        int_min = min(int_list)
        int_max = max(int_list)
        mins.append(int_min)
        maxs.append(int_max)
    # plot
    ax.fill_between(time_array, mins, maxs, color='lightgray', alpha=0.4, label='Spread')

# plot QPEs 
for i, qpe in enumerate(qpe_list):
    da = globals()[f'{qpe}_gauge_da']
    ax.plot(da.time.values, da.values, label=qpe_labels[i], 
            color=colorgroups[categories.index(category)])

# plot gauge data
ax.plot(gauge_times, gauge_intensities, label=f'Closest gauge',
        color='k', marker='.', markersize=6, lw=1.8, ls='-')

# add time of LS triggering to figure (if known)
if ls_time_known == True:
    ax.axvline(ls_time, ls='--', label='debris flow', color='darkgray')

# some things I do to make it more pretty ;)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(labelsize=16)

ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper right', fontsize=14, framealpha=0)
ax.set_xlabel('Time (UTC)', fontsize=18)
ax.set_ylabel('Hourly Precipitation Instensity (mm)', fontsize=18)

# optional - set x- and y-axis limits
#ax.set_ylim(0, intensity_max)
#ax.set_xlim(np.datetime64('YYYY-MM-DDTHH:MM'), np.datetime64('YYYY-MM-DDTHH:MM'))

if save_gauge_fig == True:
    plt.savefig('gaugeloc_time_series.png', dpi=300, transparent=True)
    
plt.show()



### plot the time series at the LANDSLIDE location

In [ ]:
# create figure
fig, ax = plt.subplots(1, 1, figsize=(18, 8))

# calculate and plot spread among QPEs at each hour
if plot_spread == True:
    mins = []
    maxs = []
    # calculate
    for hour in time_array:
        int_list = [globals()[f'{qpe}_ls_da'].sel(time=hour, method='nearest').values for qpe in qpe_list]
        int_list.append(gauge.sel(time=hour).values)
        int_min = min(int_list)
        int_max = max(int_list)
        mins.append(int_min)
        maxs.append(int_max)
    # plot
    ax.fill_between(time_array, mins, maxs, color='lightgray', alpha=0.4, label='Spread')

# plot QPEs 
for i, qpe in enumerate(qpe_list):
    da = globals()[f'{qpe}_ls_da']
    ax.plot(da.time.values, da.values, label=qpe_labels[i], 
            color=colorgroups[categories.index(category)])

# plot gauge data
ax.plot(gauge_times, gauge_intensities, label=f'Closest gauge',
        color='k', marker='.', markersize=6, lw=1.8, ls='-')

# add time of LS triggering to figure (if known)
if ls_time_known == True:
    ax.axvline(ls_time, ls='--', label='debris flow', color='darkgray')

# some things I do to make it more pretty ;)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(labelsize=16)

ax.legend(bbox_to_anchor=(1.05, 1.0), loc='upper right', fontsize=14, framealpha=0)
ax.set_xlabel('Time (UTC)', fontsize=18)
ax.set_ylabel('Hourly Precipitation Instensity (mm)', fontsize=18)

# optional - set x- and y-axis limits
#ax.set_ylim(0, intensity_max)
#ax.set_xlim(np.datetime64('YYYY-MM-DDTHH:MM'), np.datetime64('YYYY-MM-DDTHH:MM'))

if save_ls_fig == True:
    plt.savefig('lsloc_time_series.png', dpi=300, transparent=True)
    
plt.show()